# Fung data (assignee, patent) 붙여서 하나로 만들기

### patent assigned to an individual 분리 필요
- 개인이 갖고 있는 patent는 우리 관심 밖
- 그런데 Fung data의 disambiguated assignee에는 개인도 섞여 있음
- 개인 patent의 assignee를 'private'으로 바꿔주자

### Assignee 정보와 Patent 정보 붙이기
- patent 하나에 assignee 여럿일 수 있음
- 우리의 목표는 firm level로 aggregate
- 따라서 assignee 기준으로 left join

1. 기존 raw patent data에서 patent_id (5,6,7번대), private dummy 뽑음
2. 뽑은 애들이랑 Fung data iterate 하면서 assingee를 private으로 바꾸기
3. assignee와 patent patent_id 기준으로 join

### 1. 기존 raw patent data에서 id, assignee 뽑기

In [1]:
#-*- coding: utf-8 -*-
import csv
import os
import zipfile

os.chdir('E:/patent')

zf_patent1 = zipfile.ZipFile('all_patent/harvard_dataset_final_merged_output_1.zip') 
csv_patent1 = zf_patent1.open('harvard_dataset_final_merged_output_1.csv')
csv_patent_reader1 = csv.reader(csv_patent1)
header=csv_patent_reader1.next()

zf_patent2 = zipfile.ZipFile('all_patent/harvard_dataset_final_merged_output_2.zip')
csv_patent2 = zf_patent2.open('harvard_dataset_final_merged_output_2.csv')
csv_patent_reader2 = csv.reader(csv_patent2)

header2=csv_patent_reader2.next()

In [2]:
#select the assignee col
for i, name in enumerate(header):
    if 'organization_0' in name:   
        assignee_col = i
        break
        
assignee_col

19500

index: patent_id == 1, assignee == 19500  
private의 경우, assignee = 'nan'

In [3]:
with open('pid_private.csv','wb') as private:
    writer = csv.writer(private)
    header = ['patent_id','private']
    writer.writerow(header)
    
    for i in xrange(1000000):
        csv_patent_reader1.next()
    
    pid = ['5','6']    
    for line in csv_patent_reader1:
        if line[1][0] in pid and line[19500] == 'nan':
            row=[]
            row.append(line[1])
            row.append(1)
            writer.writerow(row)
    
    pid = ['6','7']    
    for line in csv_patent_reader2:
        if line[1][0] in pid and line[19500] == 'nan':
            row=[]
            row.append(line[1])
            row.append(1)
            writer.writerow(row)   
        
        elif line[1][0]=='8':
            break

### 2. private으로 바꾸기

In [4]:
#-*- coding: utf-8 -*-
import os
import csv
import pandas as pd
import sys

os.chdir('E:/')

df1 = pd.read_csv('apps/fung/assignee_1.csv')
df2 = pd.read_csv('apps/fung/assignee_2.csv')
df3 = pd.read_csv('apps/fung/assignee_3.csv')

df = df1.append([df2, df3], ignore_index=True)
del df1, df2, df3

with open('patent/pid_private.csv','rb') as prv:
    reader = csv.reader(prv)
    reader.next()
    line = reader.next()
        
    for i, row in df.iterrows():
        a=line[0]
        if row['patent_id'] == int(line[0]):
            df.loc[i,'assignee'] = 'private'
            line = reader.next()

df.to_csv('apps/fung/assignee.csv', index=False)

### 3. join

In [5]:
df1 = pd.read_csv('apps/fung/patent_1.csv')
df2 = pd.read_csv('apps/fung/patent_2.csv')
df3 = pd.read_csv('apps/fung/patent_3.csv')

df1 = df1.append([df2, df3], ignore_index=True)
del df2, df3

df1.to_csv('apps/fung/patent_vars.csv', index=False)
df = df.merge(df1, on='patent_id', how='left', sort=True)

cols = df.columns.tolist()
cols = cols[1:4] + cols[0:1] + cols[4:]
df = df[cols]

df.to_csv('apps/fung/patent_fung.csv', index=False)

In [8]:
df.to_csv('patent_fung.csv', index=False)